In [4]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [3]:
# 1. Estructura del modelo
model = DiscreteBayesianNetwork([
    ("E", "F"),
    ("E", "S"),
    ("F", "V"),
    ("S", "V"),
    ("S", "C"),
    ("V", "D")
])

# 2. CPDs

# P(E)
cpd_E = TabularCPD("E", 2, [[0.99], [0.01]])

# P(F | E)
cpd_F = TabularCPD("F", 2,
                   [[0.9, 0.4],  # F = false
                    [0.1, 0.6]],  # F = true
                   evidence=["E"], evidence_card=[2])

# P(S | E)
cpd_S = TabularCPD("S", 2,
                   [[0.95, 0.2],  # S = false
                    [0.05, 0.8]],  # S = true
                   evidence=["E"], evidence_card=[2])

# P(V | F, S)
cpd_V = TabularCPD("V", 2,
                   [[1.0, 0.5, 0.3, 0.2],  # V = false
                    [0.0, 0.5, 0.7, 0.8]],  # V = true
                   evidence=["F", "S"], evidence_card=[2, 2])

# P(C | S)
cpd_C = TabularCPD("C", 2,
                   [[0.9, 0.25],  # C = false
                    [0.1, 0.75]],  # C = true
                   evidence=["S"], evidence_card=[2])

# P(D | V)
cpd_D = TabularCPD("D", 2,
                   [[1.0, 0.4],  # D = false
                    [0.0, 0.6]],  # D = true
                   evidence=["V"], evidence_card=[2])

# 3. Agregar CPDs al modelo
model.add_cpds(cpd_E, cpd_F, cpd_S, cpd_V, cpd_C, cpd_D)

# 4. Verificar el modelo
assert model.check_model()

# 5. Inferencia: P(E = falso | D = verdadero)
infer = VariableElimination(model)
result = infer.query(variables=["E"], evidence={"D": 1})  # 1 = verdadero

# Imprimir el resultado
print(result)

+------+----------+
| E    |   phi(E) |
+======+==========+
| E(0) |   0.9361 |
+------+----------+
| E(1) |   0.0639 |
+------+----------+
